In [3]:
import sagemaker
sagemaker.__version__

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml


'2.197.0'

In [4]:
import sagemaker
import boto3
import pandas as pd
import time

sagemaker.__version__

'2.197.0'

In [7]:
!sudo apt-get update -y
!sudo curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!sudo  apt-get install git-lfs git -y

0% [Working]

Hit:1 http://us-west-2.ec2.archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://us-west-2.ec2.archive.ubuntu.com/ubuntu jammy-updates InRelease   
Hit:3 http://us-west-2.ec2.archive.ubuntu.com/ubuntu jammy-backports InRelease 
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:5 https://download.docker.com/linux/ubuntu jammy InRelease                 
Hit:6 https://deb.nodesource.com/node_16.x jammy InRelease                     
Reading package lists... Done
Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.11
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...main: line 154: /etc/apt/sources.list.d/github_git-lfs.list: Permission denied

Unable to run: 
    curl https://packagecloud.io/install/repositories/github/git-lfs/config_file.list?os=Ubuntu&dist=jammy&source=script

Double

In [16]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = "arn:aws:iam::628152409662:role/ec2-oregon-role"
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::628152409662:role/ec2-oregon-role
sagemaker bucket: sagemaker-us-east-1-628152409662
sagemaker session region: us-east-1


In [15]:
! git clone https://huggingface.co/intfloat/multilingual-e5-base

Cloning into 'multilingual-e5-base'...
remote: Enumerating objects: 44, done.
remote: Total 44 (delta 0), reused 0 (delta 0), pack-reused 44
Unpacking objects: 100% (44/44), 31.31 KiB | 866.00 KiB/s, done.
Filtering content: 100% (7/7), 3.14 GiB | 37.63 MiB/s, done.


In [17]:
!mkdir code

In [18]:
%%writefile code/inference.py

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Helper: Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def model_fn(model_dir):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # Load model from HuggingFace Hub
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModel.from_pretrained(model_dir)
    model.eval()
    model.to(device)
    return model, tokenizer

def predict_fn(data, model_and_tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # destruct model and tokenizer
    model, tokenizer = model_and_tokenizer

    # Tokenize sentences
    sentences = data.pop("inputs", data)
    
    with torch.inference_mode():

        encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        encoded_input = encoded_input.to(device)

        # Compute token embeddings
        model_output = model(**encoded_input)

        # Perform pooling
        sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings = sentence_embeddings.cpu().numpy()
        # # return dictonary, which will be json serializable
        # return {"vectors": sentence_embeddings[0].tolist()}

        # return for handling batch request
        return {"vectors": sentence_embeddings.tolist()}

Writing code/inference.py


In [19]:
repository = "intfloat/multilingual-e5-base"
model_id=repository.split("/")[-1]
s3_location=f"s3://{sess.default_bucket()}/custom_inference/{model_id}/model.tar.gz"
s3_location

's3://sagemaker-us-east-1-628152409662/custom_inference/multilingual-e5-base/model.tar.gz'

In [20]:
!cp -r code/ $model_id/code/

In [21]:
%cd $model_id
!tar zcvf model.tar.gz *

/home/ubuntu/deploy_sagemaker/multilingual-e5-base
1_Pooling/
1_Pooling/config.json
README.md
code/
code/inference.py
config.json
model.safetensors


/home/ubuntu/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


modules.json
onnx/
onnx/model.onnx
onnx/tokenizer_config.json
onnx/config.json
onnx/special_tokens_map.json
onnx/tokenizer.json
onnx/sentencepiece.bpe.model
pytorch_model.bin
sentence_bert_config.json
sentencepiece.bpe.model
special_tokens_map.json
tokenizer.json
tokenizer_config.json


In [22]:
!aws s3 cp model.tar.gz $s3_location

upload: ./model.tar.gz to s3://sagemaker-us-east-1-628152409662/custom_inference/multilingual-e5-base/model.tar.gz


In [29]:


from sagemaker.huggingface.model import HuggingFaceModel


# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=s3_location,       # path to your model and script
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version="4.12",  # transformers version used
   pytorch_version="1.9",        # pytorch version used
   py_version='py38',            # python version used
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge"
)



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml
-------------!

In [30]:
data = {
  "inputs": "Cô giáo đang dạy học",
}

res = predictor.predict(data=data)
print(res)


{'vectors': [[-0.0274589154869318, 0.04028411954641342, -0.012414602562785149, 0.04665070399641991, 0.026835940778255463, -0.04891923442482948, -0.00571406539529562, -0.02482004277408123, 0.023282432928681374, 0.016946282237768173, 0.021032800897955894, -0.014549996703863144, 0.19880668818950653, 0.05271913856267929, -0.027063626796007156, -0.03448571637272835, 0.04652881249785423, -0.009232791140675545, 0.018517930060625076, 0.023013414815068245, 0.03940308839082718, 0.013101083226501942, 0.038239601999521255, -0.03029334917664528, 0.033766407519578934, -0.013815086334943771, 0.03173404559493065, 0.003861192613840103, 0.00010607905278448015, 0.014408760704100132, 0.023689329624176025, -0.03373805433511734, 0.004887966439127922, 0.05204493924975395, 0.049225177615880966, 0.05320364609360695, -0.03525304049253464, -0.03220303729176521, 0.0842028260231018, -0.0064184521324932575, 0.008723226375877857, 0.03392680361866951, 0.021209869533777237, -0.018967969343066216, 0.030406314879655838,

In [35]:
len(res['vectors'][0])

768